In [2]:
import pandas as pd

data1 = pd.read_csv('../../raw/통합 테이블(공원)_2019.csv')
data2 = pd.read_csv('../../raw/지역사회건강조사 (전처리된 데이터)_2019/통합 테이블(건강)_2019.csv')
data3 = pd.read_csv('../../raw/서울시 주민등록인구 (연령별구별) 통계_2019_Q4.csv')
data2.rename(columns={'지역':'자치구'},inplace = True)
data3.set_index('자치구',inplace = True)
data3 = data3[['주민등록인구']]
df2 = data2[data2['연령별'] == '계']
df2 = df2[df2['성별'] == '계']
df2.set_index('자치구',inplace = True)
data1.set_index('자치구',inplace = True)
df = pd.concat([df2,data1,data3],axis=1)
df = df.drop(df.index[0])
df = df.drop(['연령별','성별'],axis=1)

In [3]:

import requests
import json
import numpy as np
from pandas.io.json import json_normalize
import os
import webbrowser
import folium
from folium import plugins
import math

In [4]:
seoul_region = '../../raw/서울시 행정구역 코드.csv'
region_data = pd.read_csv(seoul_region, encoding = 'euc-kr')
#seoul_data = state_data[['NAME','POPULATION']]
region_data.set_index('자치구',inplace = True)
total_data = pd.concat([region_data,df],axis=1)
total_data.reset_index(inplace=True)
total_data.head()

,자치구,Code,1인당 연소득,주관적 건강수준,걷기 실천,격렬한 신체활동 실천,긍정적 태도율(자연환경),주관적 스트레스 수준,우울감 경험률,중등도 신체활동 실천,...,공원면적,공원율,1인당 공원 면적,총 도시림 면적,생활권 도시림 면적,1인당 총 도시림 면적,1인당 생활권 도시림 면적,1인당 생활권 도시림 면적(산림수목),1인당 생활권 도시림 면적(공원녹지),주민등록인구
0,종로구,11110,2204.0,32.8,43.3,13.6,82.5,26.3,3.2,8.3,...,"11,556,112",48.33,71.4,13277114.30,2501869.30,87.759365,16.536911,0.639714,15.897197,161869
1,중구,11140,1988.5,43.6,65.2,24.3,82.5,21.4,2.7,9.4,...,"3,155,139",31.68,23.1,5417625.70,1506737.70,42.938755,11.942029,1.278750,10.663278,136488
2,용산구,11170,2061.0,49.3,70.1,20.8,84.7,12.8,1.8,12.7,...,"1,779,562",8.14,7.3,2005971.00,809094.00,8.772340,3.538260,2.208309,1.329951,245185
3,성동구,11200,2165.3,34.4,62.2,19.0,68.5,29.6,7.2,13.2,...,"3,100,047",18.39,10.0,1388926.00,1184497.00,4.616074,3.936658,1.076404,2.860254,308979
4,광진구,11215,1983.4,41.8,58.3,19.4,77.3,24.4,6.4,12.9,...,"3,363,481",19.71,9.2,6073160.28,1287528.28,17.285215,3.664518,1.901005,1.763512,366972


In [5]:
state_geo = '../../raw/seoul-maps-master/kostat/2013/json/seoul_municipalities_geo_simple.json'

In [6]:
json_data=open(state_geo,encoding='utf-8-sig').read()
jsonResult = json.loads(json_data)

def center_calc(points_df):
    x = points_df.x
    y = points_df.y

    X = (max(x)+min(x))/2.
    Y = (max(y)+min(y))/2.

    return X, Y

def points_array(points):

    final_points = []

    for x in range(0, len(points)):

        if len(points[x]) == 2:
            final_points.append(points[x])
        else:
            target = points[x]
            for y in range(0, len(target)):
                final_points.append(target[y])

    return final_points

center_locations = pd.DataFrame()
codes = []
regions = []
x_list = []
y_list = []

for x in range(0, len(jsonResult['features'])):
    code = jsonResult['features'][x]['properties']['code']
    name = jsonResult['features'][x]['properties']['name']

    points = jsonResult['features'][x]['geometry']['coordinates'][0]
    points = points_array(points)
    points_df = pd.DataFrame(points)
    points_df.columns = ['x','y']
    X, Y = center_calc(points_df)

    codes.append(code)
    regions.append(name)
    x_list.append(X)
    y_list.append(Y)

center_locations['Code'] = codes
center_locations['자치구'] = regions
center_locations['X'] = x_list
center_locations['Y'] = y_list

In [7]:
center_locations2 = center_locations[center_locations['Y'] >= 37.426026]
target_df = pd.merge(total_data,center_locations2, how = 'left', on = '자치구')
target_df = target_df.dropna(axis=0, subset=['X','Y']) 

In [8]:
state_geo = json.load(open('../../raw/seoul-maps-master/kostat/2013/json/seoul_municipalities_geo_simple.json',encoding='utf-8-sig'))

m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

m.choropleth(
    geo_data=state_geo,
    name='1인당 생활권 도시림 면적',
    data=total_data,
    columns=['자치구', '1인당 생활권 도시림 면적'],
    key_on='feature.properties.name',
    fill_color='Greens',
    fill_opacity=0.8,
    line_opacity=0.8,
    color = 'gray',
    legend_name = 'area'
)

for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    color = '#3186cc'

    folium.CircleMarker(location, radius=(round(target_df.iloc[i]['1인당 생활권 도시림 면적'])+10)/2,color=color,fill_color=color,fill_opacity = 0.5, opacity=0.5, 
                        popup=target_df.iloc[i]['자치구'] + "\n" + str((round(target_df.iloc[i]['1인당 생활권 도시림 면적'],2))) + "㎡/인").add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

m.save('kr_incode.html')
m

C:\Users\gimhu\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(


In [9]:

m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

m.choropleth(
    geo_data=state_geo,
    name='생활권 도시림 면적',
    data=total_data,
    columns=['자치구', '생활권 도시림 면적'],
    key_on='feature.properties.name',
    fill_color='Greens',
    fill_opacity=0.8,
    line_opacity=0.8,
    color = 'gray',
    legend_name = 'area'
)

for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    color = '#3186cc'

    folium.CircleMarker(location, radius=(target_df.iloc[i]['생활권 도시림 면적'])/1000000,color=color,fill_color=color,fill_opacity = 0.5, opacity=0.5, 
                        popup=target_df.iloc[i]['자치구'] + "\n" + str(target_df.iloc[i]['생활권 도시림 면적']) + "㎡/인").add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

m.save('kr_incode.html')
m

In [10]:
m = folium.Map(location=[37.562225, 126.978555], tiles="OpenStreetMap", zoom_start=11)

m.choropleth(
    geo_data=state_geo,
    name='주민등록인구',
    data=total_data,
    columns=['자치구', '주민등록인구'],
    key_on='feature.properties.name',
    fill_color='Reds',
    fill_opacity=0.8,
    line_opacity=0.8,
    color = 'gray',
    legend_name = 'area'
)

for i in range(0,len(target_df)):
    latitude = target_df.iloc[i]['Y']
    longitude = target_df.iloc[i]['X']
    location=(latitude, longitude)
    color = '#3186cc'

    folium.CircleMarker(location, radius=(target_df.iloc[i]['주민등록인구'])/50000,color=color,fill_color=color,fill_opacity = 0.5, opacity=0.5, 
                        popup=target_df.iloc[i]['자치구'] + "\n" + str(target_df.iloc[i]['주민등록인구']) + "㎡/인").add_to(m)


folium.LayerControl(collapsed=False).add_to(m)

m.save('kr_incode.html')
m

In [11]:
data4 = pd.read_csv('../../raw/공원위치데이터.csv')
data4 = data4.dropna(axis=0)
data4

,Unnamed: 0,관리번호,공원명,위도,경도,공원면적,자치구
0,150,11680-00061,학동공원,37.511646,127.038764,1498.5,강남구
1,151,11680-00062,신사은행나무공원,37.521114,127.021627,1240.1,강남구
2,152,11680-00063,신사목련공원,37.522898,127.017064,1001.1,강남구
5,155,11680-00066,신사개나리공원,37.526208,127.027304,991.3,강남구
6,156,11680-00067,압구정목련공원,37.532685,127.032209,2027.2,강남구
...,...,...,...,...,...,...,...
1866,17379,11560-00025,중마루근린공원,37.519919,126.911497,6456.0,영등포구
1867,17406,11560-00004,신길근린공원,37.501712,126.905408,19604.0,영등포구
1868,17433,11560-00003,문래근린공원,37.517456,126.894267,23611.0,영등포구
1869,17460,11560-00002,자매근린공원,37.516628,126.929082,16458.0,영등포구


In [12]:
map = folium.Map(location=[data4['위도'].mean(), 
                           data4['경도'].mean()],
                           zoom_start=12, tiles='OpenStreetMap')

map.choropleth(
    geo_data=state_geo,
    fill_opacity=0,
    line_opacity=0.8,
    fill_color='Greens',
    color = 'gray'
)

for n in data4.index:
    # 팝업에 들어갈 텍스를 지정해 준다.
    popup_name = data4.loc[n,'공원명']
    
    icon_color = 'green'

    
    
    folium.CircleMarker(
         location=[data4.loc[n,'위도'],data4.loc[n,'경도']],
         radius =math.sqrt(data4.loc[n,'공원면적'])/100,
         popup = popup_name,
         color=icon_color,
         fill = True,
         fill_color=icon_color     
         
    ).add_to(map)

#plugins.HeatMap(np.array([data4['위도'],data4['경도']]).T,radius=15).add_to(map) 
map.save('seoul_park.html')
map

C:\Users\gimhu\anaconda3\lib\site-packages\folium\folium.py:409: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  warnings.warn(
